IA & Data science (LU3IN0226) -- 2020-2021
--------
*&copy; Equipe pédagogique: Vincent Guigue, Christophe Marsala, Edoardo Sarti, Olivier Schwander.*


## Projet 2021

### Préparation du notebook


<font size="+1" color="RED">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

GIGACZ Kiara

## Données

Les données utilisées pour ce projet sont obtenues sur le site Kaggle, ce sont les données *Google Play Store Apps* accessibles à l'adresse https://www.kaggle.com/lava18/google-play-store-apps.


## Partie 1 - Description du problème

Le problème que nous souhaitons adresser dans ce notebook est celui de l'évaluation d'un commentaire. Pour cela, nous utilisons les colonnes "Translated_Review" et "Sentiment" du fichier "googleplaystore_user_reviews.csv".

Nous souhaitons tester si il est possible d'évaluer le sentiment d'un commentaire sans supervision. Nous pouvons utiliser la colonne sentiment pour évaluer si notre modèle fonctionne.

## Partie 2 - Modèle

Pour résoudre ce problème, nous utilisons l'algorithme des K-moyennes. 

La description d'un commentaire consiste d'un tableau de bits qui indiquent si certains mots sont présents ou non dans le nom de l'application. 

Pour obtenir ce tableau de bits, deux étapes sont nécessaires :

1. Nous faisons d'abord un traitement sur les commentaires pour obtenir un dictionnaire contenant les mots les plus fréquents. La création de ce dictionnaire est soumise à plusieurs critères que nous pouvons choisir, comme la taille minimale d'un mot ou le nombre minimum d'occurrences du mot pour qu'il soit accepté. Ces critères sont nécessaires pour réduire la taille du dictionnaire.

2. Une fois le dictionnaire obtenu, nous parcourons les commentaires avec la fonction convertText. Celle-ci crée un tableau indiquant la présence ou non de chaque mot du dictionnaire pour chaque commentaire.

## Partie 3 - Code

Importation des librairies et code à exécuter à l'ouverture du notebook.

In [2]:
# Importation des librairies standards:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

# La ligne suivante permet de préciser le chemin d'accès à la librairie iads
import sys
sys.path.append('../')

import iads as iads
from iads import Classifiers as cl
from iads import utils as ut
from iads import wordProcessing as wp

# Pour recharger automatiquement le code modifié dans les modules
%load_ext autoreload
%autoreload 2

Chargement et aperçu des données : 

In [16]:
data = pd.read_csv("../GoogleApps/googleplaystore_user_reviews.csv")
data[0:5]

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


Pré-traitement des données : 

In [17]:
# Cleaning the data

# 1. Removing rows with missing data
data = data.dropna()

data = np.array(data) # converting dataframe to ndarray
print("Data shape after removing rows with missing information", data.shape)

# 2. Removing duplicates
unique_reviews, indices = np.unique(data[:,1], return_index=True)
data = np.take(data, indices, axis=0)
print("Data shape after removing duplicates:", data.shape)

Data shape after removing rows with missing information (37427, 5)
Data shape after removing duplicates: (27994, 5)


In [41]:
textArray = data[:, 1]
minWordLength = 3
minOccurrences = 20

reviewDict = wp.createDictionary(textArray, minWordLength, minOccurrences, extraStopWords=['best', 'game', 'good', 'easy', 'like'], verbose=True)

# Save reviewDict to a txt file
keys = reviewDict.keys()
keyList = [word for word in keys]
file = open("reviewDict.txt", "w")
np.savetxt(file, keyList, "%s")
file.close()

Total number of unique words that fit requirements: 22836
Number of words that occur at least 20 times: 2546
Length of final dictionary: 130


In [42]:
convertedText = wp.convertText(textArray, reviewDict)
convertedText.shape

(27994, 130)

In [54]:
sentiments, counts = np.unique(data[:, 2], return_counts=True)
total = len(data[:, 2])
for i in range(len(sentiments)):
    print("Sentiment:", sentiments[i], "\tCount:", counts[i], "\t Percentage:", counts[i]/total)

Sentiment: Negative 	Count: 6240 	 Percentage: 0.22290490819461314
Sentiment: Neutral 	Count: 4161 	 Percentage: 0.1486389940701579
Sentiment: Positive 	Count: 17593 	 Percentage: 0.6284560977352289


## Partie 4 - Protocole expérimental

In [56]:
import timeit

k = 3
epsilon = 0.05
iter_max = 100

tic = timeit.default_timer() # heure de départ

les_centres, l_affectation = cl.kmoyennes(k, convertedText[:, :], epsilon, iter_max)

toc = timeit.default_timer() # heure d'arrivée

print(f'\nTemps mis: --> {toc-tic:.5f} secondes\n')

print("Affichage des centres: ",les_centres)
print("Affichage des affectations: ",l_affectation, "\n")

# Attention : problème potentiel quand plusieurs exemples identiques sont choisis pour l'initialisation (clusters vides)


Temps mis: --> 607.38699 secondes

Affichage des centres:  [[0.00782103 0.00454711 0.01054929 0.00727537 0.10294653 0.51891597
  0.02855584 0.00436522 0.04019644 0.0092761  0.16569662 0.02928338
  0.02237177 0.03801382 0.04401601 0.00727537 0.03528556 0.00454711
  0.0278283  0.02982903 0.00909422 0.022008   0.0058203  0.00727537
  0.03273918 0.02728265 0.07984722 0.00727537 0.01109494 0.00563841
  0.03874136 0.00381957 0.06056748 0.01491451 0.01709713 0.00727537
  0.04674427 0.06875227 0.00709349 0.03928701 0.00818479 0.01709713
  0.00472899 0.0278283  0.00618407 0.02819207 0.08439433 0.03510367
  0.0069116  0.02018916 0.01418698 0.02182612 0.0127319  0.03328483
  0.0092761  0.00200073 0.03655875 0.0209167  0.01145871 0.0823936
  0.01382321 0.00181884 0.14350673 0.0209167  0.00636595 0.02400873
  0.00763914 0.00145507 0.01345944 0.04692615 0.08039287 0.05838487
  0.00818479 0.01636959 0.0463805  0.0023645  0.0034558  0.01927974
  0.03728629 0.01582394 0.01000364 0.00291015 0.06165879 

In [57]:
positive = 0
neutral = 0
negative = 0
for i in l_affectation[0]:

    if data[i, 2] == 'Positive':
        positive += 1
    elif data[i, 2] == 'Negative':
        negative += 1
    else:
        neutral += 1   
    print(textArray[i], "\n")
    
print("Positive:", positive, "\t Neutral:", neutral, "\t Negative:", negative)
total = positive + neutral + negative
print("% Positive:", positive/total, "\t % Neutral:", neutral/total, "\t % Negative:", negative/total)

"Coming soon". You link new Google Ads app, instead. 

"Does make happy?" What kind question that? This really functional group messaging inherent ability make someone happy. The conversations take place people end conversation, makes happy. Great app. Dumb review question. 

"Fabulously Fabuloso" I wanted 2 improve quality pics & drawing,but NOW I've tried (for me),new app; I get renewed focus "photo shop", TO EDIT make lots MUCH BETTER QUALITY PICTURES THAN I EVER EXPECTED EVER;) ... So, l thank ALL OF YOU GOOGLE FRIENDS 4 IMPORTANCE OF THIS APPLICATION. Sincerely: Soto Sanchez. Paramount CA.90723. 

"I Hate's This Apps disgusting many Ads nonstop showing please remove Play store business I request 

"Mobile currently unavailable" lol ever available. The always me. I'm changing banks soon app. 

"View Medical History" Status Center work. I uninstalled installed. I force closed app. I restarted phone. Please fix bug I give 5 stars! 

(From artist perspective) This tank anatomy apps. I

A great app, I'm sure "nearby" prices - Brazil bit far travel Australia! Other that, really great app. 

A great app. I used years, gotten better time. I notes. Nice copy automatically backed up. 

A great game played anywhere, anytime. The thing need internet connection microtransactions. I think time require pay something vault goes way downhill, so, I recommend purchasing starter pack. It includes enough supplies caps get back track. Overall, I think Bethesda awesome job this, I would highly recommend it. 

A great launcher. Just 1 suggestion - home screen overview/settings mode (with 'wallpaper', 'widgets', 'settings' icons bottom screen) activated long press empty space screen standard. But stock Android also allows activated pinch action good home-screen no/little empty space long press. This option could offered 'gestures' options. Thanks. 

A great messaging app, vulnerable hacking. I've GroupMe account hacked three times, messages spammed list contacts. Other vulnerability gre


After new update.... It lags .. After hour playing ... Suddenly network lags even though strong internet connection... After min ...I would lost game... Please fix this... Happens friend phone 

After placing receiving first order, I wanted contact EatStreet let know food came early fantastic! They gave coupon $5 next order!! I officially new favorite food delivery app!! 

After recent update audio end closed. I even remove recent luck audio keeps playing. This bit frustrating, restart phone everytime close app.. 

After recent update, I delete multiple messages, messages deleted unless I switch another app, sometimes I delete messages one. Very poor performance update 

After some updates I can finally use the app, it also comes with very good graphics. 

After trying manually transfer contacts(saved different accounts) I got frustrated found app. Holy cow! It everything minutes. 

After two brain surgeries, traumatic head injury, I hate admit...im I used be. If app, I would burden s

Annoying push notifications can't disabled. The sales repeated again. Navigation issues pressing 'back' phone shopping cart view closed app. 

Another battery sapping, time wasting app. 

Another game pop ups. Give break. Fun many ads way exit them. Done games like this. 

Another half baked Google app. There way change default bids. Nothing easy find. Typical Google release app, time, start forum users get together fix Google. 

Anyway upgrade previous version Google News app? This like early 2000 design. One headline per page color scheme awful. The local news removed. It's alot cumbersome ugly design. 

Apostrophes invalid characters filenames? Really? What year this? 

App bad, an institution so rich interest costs of the population should invest in something more simple and effective, this is the slowest banking application I have ever used, every option you click takes years to open even with a great phone and 100mega connection of speed. 

App. Does work cell works desktop! I ca

Awesome app. Works excellent. 6/5 stars. I can't hear little dude crying pc sounds like 747 taking off. I work home sometimes, want buy baby monitor. Well guess what, needs anyway app. I let call landline, I hear cough he's really awake. When I hang continues listening. Perfect. 

Awesome app.. But prices date? Awesome app. Gives proper listing. Have good filter options. Also good pre-defined lists luxury, budget homes etc. But price up-to-date? I checked flat's price I checked last month. If thats good. But changed, I hope update price too. 

Awesome app..... Plz make accurat blurring....... In next update 

Awesome app❤ 

Awesome football app. You even participate game betting increasing AF points.great guys 

Awesome game. This favorite. The problem since last update, I want log game, I black screen. I always either reboot game phone...gets annoying, especially war times...and war game quite disturbing bug. 

Awesome music!!! However, I wish forest ocean tones continues play chakra 

...

Very odd definition fit What this?? Why want eat 1200 CALORIES A DAY?? 

Very poor app. Don't allow make payment, u can't type different amount minimum statement balance. Fix u want payment 

Very reliable, user-friendly convenient app. 

Very slow crashing app. Very bias articles, bad journalism. 

Very smooth app. Only add feature open multiple PDFs separate tabs. edit : So I able scan before. But latest updates I able that. It's saying separate app. Why? I also right? 

Very unstable app. Freezes time. Some planes zoom across screen 2000 MPH others dead stopped. 3D mode cool would stop freezing 10 15 seconds. I guess RAM hungry I'm using S9+ still messing up. A side note, I thought posting fake reviews Google Play policy? 90% A GOOGLE USER gave 5 stars. Seems odd. But that's observation. 

Very useful This practical young owner like me. 

Very useful app! 

Very useful app, I frequently, never problems, ask immediately...nothing more, nothing less. Wish same. 

Very useful app, allo


What disgraceful app! Delta dental needs hire real programmers create apps, I'm sure money it. Cannot even login without redirected website. 

What guys Tencent? There's folder called /tencent/Tpush/ installed root directory internal storage updated latest version. The log directory show receiving pushed contents com.tencent.andriod.tpush.action.SDK. Your shows Taiwan company, guys Tencent? I trust Chinese company all. I choice uninstall app. 

What happen app? Before I reinstalled, I can't even search flight! Showed msg : wrong method allowed. Now I cannot log account! Lousy 

What happen app? working anymore, reason I keep using truemove sim I option boost internet speed. If truemove cannot fix I continue prepaid subscription anymore transfer another carrier, maybe AIS DTac 

What happened app? Constantly crashes, asks EVERY SINGLE TIME I log activate notifications, slow, letting pay bills. This used reliable I'd ever used! 

What happened app? It used easy log exercise weight view 

Why stop working 25th May 2018? This amazing I think they're making enough money it! Please keep going 

Why update? This update take much needed internal space. If phone acting up, restart, reset, call cust. web, go nearest store help. 

Why updated app? Even I get private chat always get hang, says responding. And also I call somebody I went back users connect call I able connect , honest, update great flop, fix soon please 

Why virus program telling trojan since latest update? Seriously want know detected Trojan! Edit: sorry I uninstalled this, I risking phone called anti piracy software, reeks dishonesty, never detected Trojan before, installed fine, tells contains ransom ware.. Never happened before. 

Why work Nexus 6p? Mobile deposit can't find activate camera. Update: Even total uninstall reinstall I manually grant permissions camera items get work correctly. Once done check deposit started working. Please fix this. 

Widget nice custom color settings nothing shown data. 1x1 w

You color thing categories, 10 colors without paying. Might well make paid app. 

You disappointed weather app, right temp etc... 

You everything app. It's convenient!! 

You filter sale clothes colour "concept", size, forces go every single item see correct size. The suits listed separately can't sure trousers match jacket, filter s/m/l etc, chest trouser size. It's annoying, unintuitive awkward. Rubbish. Update 02/01/19. Still can't filter sale items. It'd quicker go H&M shop app. Still rubbish. 

You get 2 dishes 2 options cook it. You even certain ingredients. This people like wasting money 

You know games realy good. We might put ads. But done done all. So next go. And awfully funny I'm writing this, rating gone five. Hmmm. 

You must improve the integration with credit card, you can never ask for one like this 

You register purchase this! This way free app. You can't even try without registration signing free trial. Uninstalled! 

You select different countries, different lang


seems give imaginary score based scoring system actual system counts. I went bank get lone confidence credit good told score much lower app. credit lower credit check bank I'm screwed. dont trust accuracy score 

still intermittent problems. One snapshot keeps crashing app. The allow schedule payment vendors (like AMEX) next day. Forces go order accomplish this. Please fix. Alerts timely. Other banks like Chase send alerts problem happens transfer money charges incur. Citibank sends alerts next day, charged. Even catch make transfer, Citi still makes another transfer charge you. You guys better. 

still sucks buggy useless uninstalled. I'm going stick desktop version since I can't turn streaming now. Constant issue app. 

stopped working recently. Widget inaccurate. Really, bank. Invest little app! 

sucks. Doesn't work sometimes phone number store. Also save anything. I've clipped coupons never applied. I uninstalled app. 

suddenly working, start says can't connect Hamilton servers.

In [58]:
positive = 0
neutral = 0
negative = 0
for i in l_affectation[1]:

    if data[i, 2] == 'Positive':
        positive += 1
    elif data[i, 2] == 'Negative':
        negative += 1
    else:
        neutral += 1   
    print(textArray[i], "\n")
    
print("Positive:", positive, "\t Neutral:", neutral, "\t Negative:", negative)
total = positive + neutral + negative
print("% Positive:", positive/total, "\t % Neutral:", neutral/total, "\t % Negative:", negative/total)

!!!Dont waste time! Failed Samsung flagship phone galaxy s8, Installed ,shows rotating circle internet download, keeps rotates forever proper progress indication; finally shows failed download. Stupid game developers. Go NFS working good. 

"...Future Follow updated follow"... 

"An error occurred while loading the search results. Please try again." And so it's already 2 days. The reinstallation did not help 

"Cisco Spark stopped working" error group work. We 4 groups last days stopped working. Only seems issue Android devices Apple. 

"Classic" allows untimed play; game holds position interrupted put awhile. "Survival" speeds requires quick decision making. 

"Contains ads" mean adds take phone, do. 

"Dust bible leads dirt life" helps stay focused 

"Free First Phone Number" - true false advertising 

"Great innovation idea towards education highly recommendation teachers learners cheers!! " 

"ONLY A FOOL WOULD SPEND MONEY" You control wins game. What's treasure chest whole game, w


10/10 would thumbkiss again! I find new fun experience couples. At first may seem like messenger much more! The thumbkiss allows partner connect late nights long distances. Along (my personal favorite), partner sketch draw pictures play games like hangman SO! You also, create sketches send another. All all, I think fantastic couples, close 

100 inches tall color side bottom edge way get shape fit pattern hair 

100% agile 

100% fake 

100%good 

104 job hunt So amazing.. easy use.. thanx guys 

11 May update appears fixed calendar sync issue I Exchange server (where calendar sync either direction). Back five stars. 

150 ads tank okay let's get math skills back : On average ad game 30 seconds 30 x 150 750 minuites ads 12 FREAKING HOURS OF WATCHING ADS JUST TO GET A PRICELESS INGAME ITEM ARE YOU ACTUALLY INSANE? 

18sx+ 

199 % good get rid 

1st Dont think asking 5 stars 1st game almost begging.. 2nd .. game ok many pieces enough variety.. get far enough two T pieces show up.. 5 sta


A game amazing potential blocked wait hours continue story line. It's bad enough wait long energy refills energy gone mere 30 seconds. It honestly feels like devs want us play game. 

A game keeps coming back more. 

A game of beautiful Ouye and Salih Jarboha and claim 

A game, I thought suppose fun enjoyable, take weeks pass level. When open app/game know going let wino chance b4 u start, fun 

A glitches nothing can't work around. Helping weight loss complaints 

A good 

A good 'notes' Android. Colorful, fast, intuitive. It allows widgets option save lists cloud, save sync email account. One thing would make better widgets would allow scrolling. Also, I know allows collaboration lists, like 'keep' does? 

A good application, but you need to work with updates, for example an object is built, but you do not have it there 

A good camera good features 

A good easy use. I strongly recommend. 

A good game waste time bored bit I'm tryharding die something stupid, well, I'll cut chase 

AP unparalleled coverage. But there's UX thing drives batty: tapping news notification take story. If there's story, "open" button. It's frustrating. 

APFCU greatest !!! 

ASOS cuties everything. I'm 58,but still dress style. Very cool clothes,shoes,household items. The prices great. I send friends & family plus size gals. ❣❤❣ 

ASOS great quick search, always easy find looking for. 

ASSAMESE LANGUAGE WAS NOT TRANSLET WELL 

AT&T navigator Gets everywhere, sometimes signal, 95% time grest. 

ATT wants pay monthly now. No thanks. 

ATTENTION ADMINS: loved UNTIL THE LAST 2 DAYS.... I can't get load properly scans checkins.... I uninstalled reinstalled nothing changed end... 

AVG always says ,we're looking you, exactly I feel . When I'm online I'm things phone, I feel like directs away harmful material. I victim identity theft, makes feel lot comfortable things phone, like filing taxes. :-) 

AVG usually good product, unfortunately one. Besides freezing, apparently restarting freezing 


Ads nauseating, I dont care games force ads you, every level next...no thanks 

Ads ok! But too many ads. I really hate it. Uninstalling 

Ads pop-up times sensitive u accidentally easily. Even worse clicking ads, restore ro news reading previously 

Ads r much 

Ads time waste memory phone, beware every bat ad, unless course pay. No thank that's games like clash clans popular maybe company get lessons 

Ads way loud I deaf I tends bit clunky needs better programming 

Ads... 

Adsssssssss 

Advertisements cover vast majority lower navigation bar rendering almost unusable. Multiple requests support issue gone ignored. 

Advertises used it. Also seems facilitate requirements "security deposits" bookings, instead doesn't. 

Advertising 1st, game second. 

Advertising washes and washes L, the stock price falls to the top, I understand, but have to take into account the feelings of the user 

Adverts spoil game keep watching get guide line really annoying... always loved game can't play t

...


fake scammer. Every profile fake. 

family satisfied 

far best mobile game available. With new packs releasing often, daily challenges, unpredatory business model game outclasses many peers long shot. Game developers learn alot simple, amazingly made game. 

fare rate aggregator far favorite. Simple flight rankings agony great help! 

fast easy use. 

fast, lightweight, unobtrusive ads, effective, allows root browsing, better browsers, really nothing bad 

favorites lists work anymore - thought saw reviews said thing 

fcuk dumb game . laggy u cant even play . always says connection slow downloads thousands ads . u want remove ads lucky patcher. 

few options, nor can it download episodes 

filter 

fine appka, help me every day when I'm flying on the train. The current train delay is a quick search for a link including search history. Very user-friendly interface. I know I have to add, or I do not have to be so rushed. 

fine pictures short steps complete 

finger scanner set page n


great keep games keep scores everything want I highly recommend 

great last minute prices, nice hotels 

great layout soaring options 

great live 

great recipes 

great reminder order meds 

great seeing nutrition values everything, great seeing added sugar content 

great thank you ...i have omron bp device bp786n bluetooth supported pls update 

great there's problem, IOS device Sunshine club XL Android device XL version. Does XL version available IOS device Android plz help I want XL version. Plus increase capacity social spots, always users. 

great way learn shapes 

great! However, store I order - delivery driver always forgets extra sauces (that I Pay For) make second trip every time!!! 

great, I 5-star loved latest update 2 days ago. Now crashes, work. Every time I it, freezes closes. 

great, I like get explanation question get wrong. 

great, like I a,ways imagined weakness becomes med staff data entry procedures/policies e.g., Home Care notes, health group IT limitation


like it, nice clothes kfashion 

like kind felt like I inside tunnel I call phone. Other seems work pretty good I like I able expand screen see people's screens even though I phone that's nice. 

like thhhiiss game lot :D 

like, allegedly, WHAT fixed??? disclose, please. post acurate complete CHANGELOG... theater/seating data randomly incorrect... fast expiring points tantamount bait-and-switch "benefit." still keep seeing people surprising Fandango "glitches" showtime ticket taking... 

liked 

liked dis game 

little short becoming greatest - Must haves- 1. Ability add word, pdf, ppt docs directly notes. And able search words within them. 2. S-Pen writing features could enhanced. 3. List view could added displays note titles. 4. The ability take notes offline case limited / cellular connectivity would great feature. 5. Ability crop / edit photos. 6. Crop photos within note. 7.Add Templates tables filled hand writing / s-pen / stylus. 8. Auto detect correct manual drawing Geometrica


please improve hitwe issue wrong country code passed? known issue plzz merit karny kafi dino se yehi hai chal ri 

please someone answer me!!! can't add photo tip?????????? 

plz create indian people 

post Facebook twitter since Aug. 1 2018. Newest version broken. 

pretty bad around. The worst google pixel xl 2 I even player tab see available players league signs right out. Despite fully updated uninstalled/installed. Very disappointed 

pretty good, listening music feels good. But noise disturbed again. Do something noise. 

pretty solid decided make play money became apps. Additionally, code got worse would create code events selected. It went 4.5 2.5 

previous version functioning well. live news section working all. uninstall it. 

private contacts snoop messages like asus zenfone max zc550kl 

problem remember login ID password . 

problem, now, sudden there's message saying weather unavailable. There's problem internet connection though. 

problems game global chat like clash 

travel information place, convenient. add features check directly book flights/hotels. 

tried searching movies wanted watch trailers even though said full movie 

tried several text bubble easiest use. 

ts best thing ever except BTS 

tttii 

turns book sideways soon opens new update. Completely unusable. Garbage. 

u put date couldnt change return date u come go date goofy 

ui is better than other Nevi apps. It is very familiar with the sound of a long time using the navigation system dedicated to navigation. 

unable get past terms agreement 

understand Chinese, you. 

unreliable. randomly forgets reminders, completely useless that. uninstalled. 

unstable freezes often 

unusable. lags. tons ads . turns logging meal complicated frustrating procedure. hordid 

update came actually remove people "likes me" tab I decide match them. kind annoying see gym selfies pop I've said no. otherwise fine. nice able set deal breakers see people respond various prompts. 

update version 4.26.7 

## Partie 5 - Résultats

Les clusterings obtenus sont souvent basés sur la présence d'un mot (s'il est présent alors le commentaire appartient à un cluster, sinon il appartient à l'autre cluster).

Par exemple, les mots pour lesquels ce phénomène se produit sont : game, best, like, good, easy.

Quand ces mots sont retirés (rajoutés à la liste des "stopwords"), la catégorisation est plus difficile à évaluer.


## Partie 6 - Analyse

L'algorithme ne parvient pas à trouver des clusterings intéressants pour notre problème sans supervision. Néanmoins, il est capable de trouver les mots clés qui sont très populaires.